In [1]:
import numpy as np
from src.helpers import load_csv_data, standardize
from src.implementations import least_squares
from src.costs import compute_loss

In [2]:
y, tx, ids = load_csv_data('data/test.csv')

tx, x_mean, x_std = standardize(tx)

In [3]:
def build_poly_matrix(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    
    powers = np.tile(np.tile(np.arange(degree + 1), x.shape[1]), (x.shape[0], 1))
    expanded_x = np.repeat(x, degree + 1, axis=1)
    
    return np.power(expanded_x, powers)

In [4]:
test = np.random.randint(10, size=(3, 5))
print(test)
print(build_poly_matrix(test, 2))

[[8 7 3 2 9]
 [9 0 7 4 2]
 [0 6 5 1 6]]
[[ 1  8 64  1  7 49  1  3  9  1  2  4  1  9 81]
 [ 1  9 81  1  0  0  1  7 49  1  4 16  1  2  4]
 [ 1  0  0  1  6 36  1  5 25  1  1  1  1  6 36]]


In [8]:
def polynomial_regression():
    """Constructing the polynomial basis function expansion of the data,
       and then running least squares regression."""
    # define parameters
    degrees = [2]

    for ind, degree in enumerate(degrees):
        
        # Form the data to do polynomial regression
        phi_x = build_poly_matrix(tx, degree)
        print(tx.shape)
        print(phi_x.shape)

        # Least square and calculate RMSE
        mse, weights = least_squares(y, phi_x)
        
        rmse = np.math.sqrt(2 * mse)

        print("Processing {i}th experiment, degree={d}, rmse={loss}".format(
              i=ind + 1, d=degree, loss=rmse))

polynomial_regression()

(568238, 30)
(568238, 90)


LinAlgError: Singular matrix

In [5]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

In [6]:
def compute_rmse(y, x, w):
    l = compute_loss(y, x, w)
    return np.math.sqrt(2*l)

In [7]:
from src.implementations import ridge_regression
from src.polynomials import build_poly_matrix

def cross_validation(y, x, k_indices, k, lambda_, degree, mean=True):
    """return the loss of ridge regression."""
    # Get k'th subgroup in test, others in train
    
    losses_tr, losses_te, ws = [], [], []
    
    for k_ in range(k):
        
        test_indices = k_indices[k_]
        train_indices = np.setdiff1d(k_indices.flatten(), test_indices)

        y_train = y[train_indices]
        x_train = x[train_indices]
        y_test = y[test_indices]
        x_test = x[test_indices]

        # Form data with polynomial degree
        x_train_poly = build_poly_matrix(x_train, degree)
        x_test_poly = build_poly_matrix(x_test, degree)

        # Ridge regression
        loss_tr, w_ridge = ridge_regression(y_train, x_train_poly, lambda_)

        # Calculate the loss for test data
        loss_te = compute_loss(y_test, x_test_poly, w_ridge)
        
        losses_tr.append(np.math.sqrt(2 * loss_tr))
        losses_te.append(np.math.sqrt(2 * loss_te))
        ws.append(w_ridge)
    
        
        
    return np.mean(losses_tr), np.mean(losses_te)

In [ ]:
from src.plots import cross_validation_visualization
import time

def cross_validation_demo():
    seed = 1
    degree = 7
    k_fold = 4
    lambdas = np.logspace(-4, 0, 30)
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)
    # define lists to store the loss of training data and test data
    rmse_tr, rmse_te = [], []
    
    
    start_time = time.time()
    
    for lambda_ in lambdas:        
        
        loss_tr, loss_te = cross_validation(y, tx, k_indices, k_fold, lambda_, degree)
        rmse_tr.append(loss_tr)
        rmse_te.append(loss_te)
    
    print(time.time() - start_time)
        
    cross_validation_visualization(lambdas, rmse_tr, rmse_te)

%timeit cross_validation_demo()